In [5]:
import pandas as pd
import json
import streamlit as st
import wget
import sqlite3

### captura dos dados

In [44]:
countries = pd.read_json('https://gco.iarc.fr/causes/obesity/data/countries.json')
cancers = pd.read_json('https://gco.iarc.fr/causes/obesity/data/cancers.json')

In [45]:
#attributable = wget.download('https://gco.iarc.fr/causes/obesity/data/attributable.json', out='../data/attributable.json')
#preventable = wget.download('https://gco.iarc.fr/causes/obesity/data/preventable.json', out='../data/preventable.json')

In [6]:
with open('../data/attributable.json') as json_data:
    data = json.load(json_data)
    attributable = pd.json_normalize(data['both'])

In [7]:
with open('../data/preventable.json') as json_data:
    data = json.load(json_data)
    preventable = pd.json_normalize(data['both'])

In [8]:
attributable = attributable[['total_m', 'total_f', 'country.name', 'country.WHO_NAME', 'cancer.name', 'paf_obs_m', 'paf_obs_f', 'paf_obs_both']]
attributable = attributable.rename(columns={"country.name":"country", "country.WHO_NAME":"continent", "cancer.name":"cancer"})
attributable['continent'] = attributable['continent'].apply(lambda x: x.replace("WHO ",""))

In [50]:
preventable = preventable[['total_m', 'total_f', 'country.name', 'country.WHO_NAME', 'cancer.name', 'paf_obs_m', 'paf_obs_f', 'paf_obs_both']]
preventable = preventable.rename(columns={"country.name":"country", "country.WHO_NAME":"continent", "cancer.name":"cancer"})
preventable['continent'] = preventable['continent'].apply(lambda x: x.replace("WHO ",""))

In [35]:
con = sqlite3.connect("../data/tutorial.db")
join = pd.read_sql('select a.country from attributable a left join preventable p on a.country = p.country', con=con)

In [49]:
attributable.head()

,total_m,total_f,country,continent,cancer,paf_obs_m,paf_obs_f,paf_obs_both
0,3464,5653,Angola,Africa region,Oesophageal adenocarcinoma,0.101688,0.184468,0.131454
1,3464,5653,Angola,Africa region,Pancreas,0.023512,0.033597,0.027949
2,3464,5653,Angola,Africa region,Kidney,0.042687,0.107470,0.077496
3,3464,5653,Angola,Africa region,Colon,0.035932,0.031838,0.033761
4,3464,5653,Angola,Africa region,Rectum,0.017498,0.015740,0.016561


In [53]:
con = sqlite3.connect("../data/tutorial.db")

In [54]:
attributable.to_sql('attributable', con=con, if_exists='replace')
preventable.to_sql('preventable', con=con, if_exists='replace')

1584